In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
year_html = None #########

In [3]:
def soup(year):
    
    url = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
    
    print(f"Asking for http content from {url}")
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    
    print(f"Filtering by year {year}...")
    global year_html
    year_html = soup.find_all("div", id=f"faq{year}")[0] ##############
    print(f"html generated")
    
    ########return years

In [4]:
def df(year, month):
    ###########year_html = soup(year)
    months_available = [elem.get_text(strip=True) for elem in year_html.find_all(['b'])]
    
    print(f"Checking month requested {month} vs months available in the html content...")
    if month not in months_available:
        raise ValueError(f"month provided does not match with available months, check spelling {months_available}")
    
    print(f"Month provided is valid. Filtering by month...")
    filter_by_month = year_html.find(string=f"{month}").parent.findNext('ul')
    links = [elem.a['href'] for elem in filter_by_month.find_all('li')]
    description = [elem.a.text for elem in filter_by_month.find_all('li')]
    
    final_df = pd.DataFrame({'month': month, 'links': links, 'description': description})
    
    pd.set_option('display.max_colwidth', None)
    
    return final_df

In [5]:
#df(2020, 'January')

In [6]:
def create_links_table(year, *month):
    
    links = pd.DataFrame()
    
    for elem in month:
        links = links.append(df(year, elem), ignore_index=True)
 
    print("Generating csv links table")
    
    return links
    

In [7]:
#result = create_links_table(2020, 'January', 'February', 'March')

In [8]:
#result

In [9]:
#csv = result[result['description'].str.contains('Yellow')]['links']#['links'][0]
#csv

def csv_to_df(year, taxi_color ,*month):
    
    soup(year) ###
    
    print(f"list of months requested: {month}")
    links_table = create_links_table(year, *month)
    links_table = links_table[links_table['description'].str.contains(taxi_color)]['links']
    print(links_table)
    
    final_df = pd.DataFrame()
    
    for elem in links_table:
        print(f"Reading csv {elem}")
        final_df = final_df.append(pd.read_csv(elem), ignore_index=True)
        print("Completed")
    

    type_object = final_df.select_dtypes(include='object').columns.to_list()
    type_object.pop()
    
    for elem in type_object:
        final_df[f"{elem}"] = pd.to_datetime(final_df[f"{elem}"], yearfirst=True, format="%Y/%m/%d")
    
    name = str(2020) + '_' + ','.join(month).replace(',', '_') + '_' + f'{taxi_color}' + '.csv'
    final_df.to_csv(name, index=False)
    print("csv generated")
    
    #return final_df
    

In [12]:
def csv_to_df(year, taxi_color ,*month):
    
    soup(year) ###
    
    print(f"list of months requested: {month}")
    links_table = create_links_table(year, *month)
    links_table = links_table[links_table['description'].str.contains(taxi_color)]['links']
    print(links_table)
    
    #final_df = pd.DataFrame()
    
    for elem in links_table:
        print(f"Reading csv {elem}")
        single_df = pd.read_csv(f"{elem}")
        type_object = single_df.select_dtypes(include='object').columns.to_list()
        type_object.pop()
        for typ in type_object:
            single_df[f"{typ}"] = pd.to_datetime(single_df[f"{typ}"], yearfirst=True, format="%Y/%m/%d")
        
        name = str(2020) + '_' + elem + '_' + f'{taxi_color}' + '.csv'
        
        single_df.to_csv(name, index=False)
        
        print("Completed")
    

    print("csv generated")
    
    #return final_df

In [13]:
csv_to_df(2020, 'Yellow', 'January', 'February', 'March')

Asking for http content from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
Filtering by year 2020...
html generated
list of months requested: ('January', 'February', 'March')
Checking month requested January vs months available in the html content...
Month provided is valid. Filtering by month...
Checking month requested February vs months available in the html content...
Month provided is valid. Filtering by month...
Checking month requested March vs months available in the html content...
Month provided is valid. Filtering by month...
Generating csv links table
0    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.csv
4    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-02.csv
8    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-03.csv
Name: links, dtype: object
Reading csv https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.csv


ImportError: Missing optional dependency 'fsspec'.  Use pip or conda to install fsspec.

In [ ]:
#full_csv = pd.read_csv('2020_January_February_March_Yellow.csv')
#full_csv

In [ ]:
#full_csv.dtypes

In [ ]:
#full_csv.head()

In [ ]:
#full_csv.tail()